In [1]:
import numpy as np


In [2]:
from keras.callbacks import ModelCheckpoint
from keras.layers import Activation, Dense, Dropout, Embedding, LSTM, TimeDistributed, Flatten
from keras.models import Sequential
from keras.preprocessing import sequence
from keras.utils import np_utils


Using TensorFlow backend.


In [8]:
SEQ_LENGTH = 100
length = 250
HIDDEN_DIM = 1000
BATCH_SIZE = 50
LAYER_NUM = 3
EPOCHS = 30


In [4]:
print('Reading data...') 
FILENAME = 'fortune_quotes.txt' 
data = open(FILENAME, 'r', encoding='utf8').read()

Reading data...


In [5]:
print('Creating dictionary...')

data = data.replace('/', '').replace('\r', '').replace(
    '"', '').replace('[', '').replace(']', '').replace(
    '“', '').replace('”', '').replace(':', '').replace(
    '(', '').replace(')', '').replace('\x9d', '').replace(
    '\ufeff', '').replace('â', '').replace(
    '™', '').replace('œ', '').replace('¦', '').replace(
    '`', '').replace('\n', ' ')
chars = list(set(data))
VOCAB_SIZE = len(chars)

ix_to_char = {ix:char for ix, char in enumerate(chars)}
char_to_ix = {char:ix for ix, char in enumerate(chars)}

print('Formatting data for analysis...')

X = np.zeros((len(data)//SEQ_LENGTH, SEQ_LENGTH, VOCAB_SIZE))
y = np.zeros((len(data)//SEQ_LENGTH, SEQ_LENGTH, VOCAB_SIZE))
for i in range(0, len(data)//SEQ_LENGTH):
    X_sequence = data[i*SEQ_LENGTH:(i+1)*SEQ_LENGTH]
    X_sequence_ix = [char_to_ix[value] for value in X_sequence]
    input_sequence = np.zeros((SEQ_LENGTH, VOCAB_SIZE))
    for j in range(SEQ_LENGTH):
        input_sequence[j][X_sequence_ix[j]] = 1.
    X[i] = input_sequence

    y_sequence = data[i*SEQ_LENGTH+1:(i+1)*SEQ_LENGTH+1]
    y_sequence_ix = [char_to_ix[value] for value in y_sequence]
    target_sequence = np.zeros((SEQ_LENGTH, VOCAB_SIZE))
    for j in range(SEQ_LENGTH):
        target_sequence[j][y_sequence_ix[j]] = 1.
    y[i] = target_sequence

print('Finished!  Parsed, cleaned and formatted \
a document with {:,} total characters and {:,} \
unique characters for analysis.'.format(len(data), len(chars),))


Creating dictionary...
Formatting data for analysis...
Finished!  Parsed, cleaned and formatted a document with 824,776 total characters and 76 unique characters for analysis.


In [6]:
def generate_text(model, length):
    ix = [np.random.randint(VOCAB_SIZE)]
    y_char = [ix_to_char[ix[-1]]]
    X = np.zeros((1, length, VOCAB_SIZE))
    for i in range(length):
        X[0, i, :][ix[-1]] = 1
        print(ix_to_char[ix[-1]], end="")
        ix = np.argmax(model.predict(X[:, :i+1, :])[0], 1)
        y_char.append(ix_to_char[ix[-1]])
    return ('').join(y_char)

In [7]:
model = Sequential()
model.add(LSTM(HIDDEN_DIM, input_shape=(None, VOCAB_SIZE), return_sequences=True))
model.add(LSTM(HIDDEN_DIM, return_sequences=True))
for i in range(LAYER_NUM - 1):
    model.add(LSTM(HIDDEN_DIM, return_sequences=True))
model.add(TimeDistributed(Dense(VOCAB_SIZE)))
model.add(Activation('softmax'))
model.compile(loss="categorical_crossentropy", optimizer="rmsprop", metrics=['accuracy'])
print('\n')
print('Model Ready for Training...')
print('\n')



Model Ready for Training...




In [9]:
print("Training started...")
print('\n')
for i in range(EPOCHS):
    print('INFO - Training model: Epoch: ', i+1, ' / ', EPOCHS)
    model.fit(X,
              y,
              batch_size=BATCH_SIZE,
              epochs=1,
              verbose=1,
              shuffle=False)
    generate_text(model, length)
    print('\n')
    if (i % 10 == 0):
        model.save_weights('checkpoint_{}_epoch_{}.hdf5'.format(HIDDEN_DIM, i))
        
print('Training Complete!')


Training started...


INFO - Training model: Epoch:  1  /  30
Epoch 1/1
8247/8247 [==============================] - 99s - loss: 3.1035 - acc: 0.1708    
9t                                                                                                                                                                                                                                                        

INFO - Training model: Epoch:  2  /  30
Epoch 1/1
8247/8247 [==============================] - 97s - loss: 2.8585 - acc: 0.2053    
Le an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an a

INFO - Training model: Epoch:  3  /  30
Epoch 1/1
8247/8247 [==============================] - 96s - loss: 2.3446 - acc: 0.3159    
L You whet You whet you whet you whet you whet you whet you whet you whet you 

8247/8247 [==============================] - 97s - loss: 0.4951 - acc: 0.8487    
y is a virtue, but not when it is in yet will all care able to your problems. You will always have good luck in your paces. Your life will accomplish sound how to get there and purit formed one idea. Your life will get more and more exciting. Your lo

INFO - Training model: Epoch:  23  /  30
Epoch 1/1
8247/8247 [==============================] - 97s - loss: 0.4454 - acc: 0.8645    
4 ask you long door at the time of the tide. You will think you are working hard today. You are going to be a person. You love it! You love and advertising. You will take a chance in something in near future. You look at all the time. You will take a

INFO - Training model: Epoch:  24  /  30
Epoch 1/1
8247/8247 [==============================] - 97s - loss: 0.3958 - acc: 0.8799    
&will take a pleasant. Your heart will always make itself gets yourself. Your emotional currents are always really. Your ability will be an inspirat

In [11]:
generate_text(model, length)

X find in the day to a short row. You will be sharing great news with all people you love through a year. You will be sharing great news with all people you love You will be successful in your work. You will be sharing great news with a book where yo

'X find in the day to a short row. You will be sharing great news with all people you love through a year. You will be sharing great news with all people you love You will be successful in your work. You will be sharing great news with a book where you'